# Natural language processing concepts with spaCy

By [Allison Parrish](http://www.decontextualize.com/)

“Natural Language Processing” is a field at the intersection of computer science, linguistics and artificial intelligence which aims to make the underlying structure of language available to computer programs for analysis and manipulation. It’s a vast and vibrant field with a long history! New research and techniques are being developed constantly.

The aim of this notebook is to introduce a few simple concepts and techniques from NLP—just the stuff that’ll help you do creative things quickly, and maybe open the door for you to understand more sophisticated NLP concepts that you might encounter elsewhere. We'll start with simple extraction tasks: isolating words, sentences, and parts of speech. By the end, we'll have a few working systems for creating sophisticated text generators that function by remixing texts based on their constituent linguistic units. This tutorial is written for Python 3.6+.

There are a number of libraries for performing natural language processing tasks in Python, including:

* [NLTK](http://www.nltk.org/): a workhorse of a library, still widely used, but somewhat dated in terms of capabilities
* [Stanza](https://stanfordnlp.github.io/stanza/index.html) is a newish natural language processing library for Python, developed by the storied [Stanford NLP Group](https://nlp.stanford.edu/). It includes a client library for Stanford NLP's [CoreNLP library](https://stanfordnlp.github.io/CoreNLP/) for functionality not yet natively available in Python.
* [AllenNLP](https://allennlp.org/) is a library for developing and deploying natural language processing machine learning models, including models for [sentiment analysis](https://demo.allennlp.org/sentiment-analysis), [constituency parsing](https://demo.allennlp.org/constituency-parsing) and [co-reference resolution](https://demo.allennlp.org/coreference-resolution).

But we'll be using a library called [spaCy](https://spacy.io/), which very powerful and easy for newcomers to understand. It's been among the most important tools in my text processing toolbox for many years!


## Natural language

“Natural language” is a loaded phrase: what makes one stretch of language “natural” while another stretch is not? NLP techniques are opinionated about what language is and how it works; as a consequence, you’ll sometimes find yourself having to conceptualize your text with uncomfortable abstractions in order to make it work with NLP. (This is especially true of poetry, which almost by definition breaks most “conventional” definitions of how language behaves and how it’s structured.)

Of course, a computer can never really fully “understand” human language. Even when the text you’re using fits the abstractions of NLP perfectly, the results of NLP analysis are always going to be at least a little bit inaccurate. But often even inaccurate results can be “good enough”—and in any case, inaccurate output from NLP procedures can be an excellent source of the sublime and absurd juxtapositions that we (as poets) are constantly in search of.

## Language support

Historically, most NLP researchers have focused their efforts on English specifically. But many natural language processing libraries now support a wide range of languages. You can find the full [list of supported languages](https://spacy.io/usage/models#languages) on their website, though the robustness of these models varies from one language to the next, as does the specifics of how the model works. (For example, different languages have different ideas about what a "part of speech" is.) The examples in this notebook are primarily in English. If you're having trouble applying these techniques to other languages, send me an e-mail—I'd be happy to help you figure out how to get things working for languages other than English!

## English grammar: a crash course

The only thing I believe about English grammar is [this](http://www.writing.upenn.edu/~afilreis/88v/creeley-on-sentence.html):

> "Oh yes, the sentence," Creeley once told the critic Burton Hatlen, "that's
> what we call it when we put someone in jail."

There is no such thing as a sentence, or a phrase, or a part of speech, or even
a "word"---these are all pareidolic fantasies occasioned by glints of sunlight
we see on reflected on the surface of the ocean of language; fantasies that we
comfort ourselves with when faced with language's infinite and unknowable
variability.

Regardless, we may find it occasionally helpful to think about language using
these abstractions. The following is a gross oversimplification of both how
English grammar works, and how theories of English grammar work in the context
of NLP. But it should be enough to get us going!

### Sentences and parts of speech

English texts can roughly be divided into "sentences." Sentences are themselves
composed of individual words, each of which has a function in expressing the
meaning of the sentence. The function of a word in a sentence is called its
"part of speech"—i.e., a word functions as a noun, a verb, an adjective, etc.
Here's a sentence, with words marked for their part of speech:

    I       really love entrees       from        the        new       cafeteria.
    pronoun adverb verb noun (plural) preposition determiner adjective noun

Of course, the "part of speech" of a word isn't a property of the word itself.
We know this because a single "word" can function as two different parts of speech:

> I love cheese.

The word "love" here is a verb. But here:

> Love is a battlefield.

... it's a noun. For this reason (and others), it's difficult for computers to
accurately determine the part of speech for a word in a sentence. (It's
difficult sometimes even for humans to do this.) But NLP procedures do their
best!

### Phrases and larger syntactic structures

There are several different ways for talking about larger syntactic structures in sentences. The scheme used by spaCy is called a "dependency grammar." We'll talk about the details of this below.

## Installing spaCy

There are [instructions for installing spaCy](https://spacy.io/usage) on the spaCy web page. You can also install it by running the following cell in this notebook:

In [ ]:
import sys
!conda install -c conda-forge -y --prefix {sys.prefix} spacy

You'll also need to download a language model. You can download the default language model for English by running the cell below:

In [ ]:
import sys
!{sys.executable} -m spacy download en_core_web_md

Replace `en_core_web_md` with the name of the model you want to install. [The spaCy documentation explains the difference between the various models](https://spacy.io/models).

The language model contains machine learning models for splitting texts into sentences and words, tagging words with their parts of speech, identifying entities, and discovering the syntactic structure of sentences.

## Basic usage

Import `spacy` like any other Python module:

In [280]:
import spacy

Create a new spaCy object using `spacy.load(...)`. The name in the parentheses is the same as the name of the model you downloaded above. If you downloaded a different model, you can put its name here instead.

In [281]:
nlp = spacy.load('en_core_web_md')

It's more fun doing natural language processing on text that you're interested in. I recommend grabbing a something from [Project Gutenberg](http://www.gutenberg.org/). Download a plain text file and put it in the same directory as this notebook, taking care to replace the filename in the cell below with the name of the file you downloaded.

In [282]:
# replace "84-0.txt" with the name of your own text file
text = open("allbillsnostopwords.txt").read()

Now, use spaCy to parse it. (This might take a while, depending on the size of your text.)

In [283]:
nlp.max_length = 5000000 #or any large value, as long as you don't run out of RAM
doc = nlp(text)

Right off the bat, the spaCy library gives us access to a number of interesting units of text:

* All of the sentences (`doc.sents`)
* All of the words (`doc`)
* All of the "named entities," like names of places, people, #brands, etc. (`doc.ents`)
* All of the "noun chunks," i.e., nouns in the text plus surrounding matter like adjectives and articles

The cell below, we extract these into variables so we can play around with them a little bit.

In [284]:
nlp.add_pipe('sentencizer')

In [285]:
sentences = list(doc.sents)
words = [w for w in list(doc) if w.is_alpha]
noun_chunks = list(doc.noun_chunks)
entities = list(doc.ents)

### Counting and sampling

With this information in hand, we can answer interesting questions like: how many sentences are in the text?

In [286]:
len(sentences)

18949

Using `random.sample()`, we can get a small, randomly-selected sample from these lists. Here are five random sentences:

In [287]:
import random
for item in random.sample(sentences, 5):
    print(item.text.strip().replace("\n", " "))
    print()

SECTION 4.

5 SEC.

enacted General Assembly State Missouri, follows: Section A. Chapter 170, RSMo, amended adding thereto one new section, known section 170.385, read follows:3 170.385.

damages shall cumulative way limited remedies may available federal, state, municipal law.

event hearing hearings officials receiving state, governing law shall sending state record hearing hearings prescribed sending state shall made.



Ten random words:

In [288]:
for item in random.sample(words, 10):
    print(item.text)

otherwise
administrative
monetary
Sex
school
circumstances
verifiable
STUDENTS
cartilage
relating


Ten random noun chunks:

In [289]:
for item in random.sample(noun_chunks, 10):
    print(item.text)

power school district board directors
parents
student's sex
Violation(s
17 interest child
physiological differences
person
foster parents
another person
.


Ten random entities:

In [290]:
import re

In [291]:
for item in random.sample(entities, 10):
    print(item.text)

4:09:50 PM
two
37
27
annually
5
3
1
Idaho
1


### spaCy data types

Note that the values that spaCy returns belong to specific spaCy data types. You can read more about [these data types](https://spacy.io/api) in the spaCy documentation, in particular [spans](https://spacy.io/api/span/) and [tokens](https://spacy.io/api/token). (Spans represent sequences of tokens; a sentence in spaCy is a span, and a word is a token.) If you want a list of strings instead of a list of spaCy objects, use the `.text` attribute, which works for spans and tokens alike. For example:

In [292]:
sentence_strs = [item.text for item in doc.sents]

In [293]:
random.sample(sentence_strs, 10)

['(2) person shall knowingly engage conduct aids abets gender reassignment surgery services performed provided minor.',
 '6 act would take effect January 1, 2024.',
 '33 1. public fund shall following: 34 a. Grant proxy voting authority person 35 -3- LSB 1119YH (6) 90 ec/rn 3/ 10 H.F. 2 part public fund adopt practice following 1 recommendations proxy advisor unless person 2 advisor practice of, writing commits to, 3 following guidelines act solely financial interest 4 participants beneficiaries public fund based 5 upon improper financial factors.',
 "(c) school district adopts policy allows separation multi-occupancy bathrooms changing facilities according criterion person's biological sex assigned birth, policy must: (1) include: (A) description of: (i) multi-occupancy bathrooms changing facilities separated biological sex assigned birth; (ii) multi-occupancy bathroom changing facility separated biological sex assigned birth, method students supervised facility manner method supervis

### Parts of speech

The spaCy parser allows us to check what part of speech a word belongs to. In the cell below, we create four different lists—`nouns`, `verbs`, `adjs` and `advs`—that contain only words of the specified parts of speech. ([There's a full list of part of speech tags here](https://spacy.io/docs/usage/pos-tagging#pos-tagging-english)).

In [294]:
nouns = [w for w in words if w.pos_ == "NOUN"]
verbs = [w for w in words if w.pos_ == "VERB"]
adjs = [w for w in words if w.pos_ == "ADJ"]
advs = [w for w in words if w.pos_ == "ADV"]

And now we can print out a random sample of any of these:

In [295]:
for item in random.sample(advs, 20): # change "nouns" to "verbs" or "adjs" or "advs" to sample from those lists!
    print(item.text)

otherwise
explicitly
right
normally
accurately
directly
physically
otherwise
immediately
also
Sexually
irresolvably
also
Whereas
immediately
commercially
naturally
hereby
generally
sincerely


### Entity types

The parser in spaCy not only identifies "entities" but also assigns them to a particular type. [See a full list of entity types here.](https://spacy.io/docs/usage/entity-recognition#entity-types) Using this information, the following cell builds lists of the people, locations, and times mentioned in the text:

In [296]:
people = [e for e in entities if e.label_ == "PERSON"]
locations = [e for e in entities if e.label_ == "LOC"]
times = [e for e in entities if e.label_ == "TIME"]
GPE = [e for e in entities if e.label_ == "GPE"]

And then you can print out a random sample:

In [297]:
for item in random.sample(GPE, 20): # change "times" to "people" or "locations" to sample those lists
    print(item.text.strip())

Hawaii
R.I.
Kansas
U.S.
Harbin
United States
Dist 9
New York
Arkansas
Arkansas
county
Minnesota
LB810
F.S.
county
Hawaii
United States
LUCERO
Arkansas
Kentucky


## Finding the most common

After we've parsed the text out into meaningful units, it might be interesting to see which examples of those units are the most common in a text.

One of the most common tasks in text analysis is counting how many times things occur in a text. The easiest way to do this in Python is with the [`Counter` object, contained in the `collections` module](https://docs.python.org/3/library/collections.html#collections.Counter). Run the following cell to create a `Counter` object to count your words.

In [298]:
from collections import Counter
word_count = Counter([w.text for w in words])

Once you've created the counter, you can check to see how many times any word occurs like so:

In [299]:
word_count['subjective']

65

The `Counter` object's `.most_common()` method gives you access to a list of tuples with words and their counts, sorted in reverse order by count:

In [300]:
word_count.most_common(10)

[('shall', 5194),
 ('school', 4537),
 ('section', 3027),
 ('sex', 2982),
 ('health', 2702),
 ('person', 2662),
 ('child', 2541),
 ('state', 2367),
 ('may', 2359),
 ('care', 2250)]

The code in the following cell prints this out nicely:

In [301]:
for word, count in word_count.most_common(40):
    print(word, count)

shall 5194
school 4537
section 3027
sex 2982
health 2702
person 2662
child 2541
state 2367
may 2359
care 2250
public 2088
gender 2075
means 2025
student 1810
Section 1647
medical 1607
individual 1605
parent 1587
SECTION 1511
action 1458
district 1415
a 1397
including 1394
b 1356
law 1290
board 1256
biological 1240
act 1167
provided 1149
subsection 1062
minor 1046
services 1037
Code 1036
violation 1019
education 965
years 960
c 926
sexual 924
Act 921
institution 908


You'll note that the list of most frequent words here likely reflects the overall frequency of words in English. Consult my [Quick and dirty keywords](quick-and-dirty-keywords.ipynb) tutorial for some simple strategies for extracting words that are most unique to a text (rather than simply the most frequent words). You may also consider [removing stop words](https://stackabuse.com/removing-stop-words-from-strings-in-python/) from the list.

In [302]:
from math import e
def expected(nlp, word, assumed_total=1e7):
    guess = (e**nlp.vocab[word].prob) * assumed_total
    return (guess, assumed_total)

In [303]:
observed = Counter([item.text for item in doc])
observed_total = sum(observed.values())

In [304]:
from scipy.stats import chi2_contingency
chi2_contingency([(observed["gender"], observed_total), expected(nlp, "gender")], lambda_=0)
def spacygtest(nlp, ob_counts, ob_total, word, assumed_total=1e7):
    g = chi2_contingency([[ob_counts[word], ob_total], expected(nlp, word)], lambda_=0)
    return g[0]
spacygtest(nlp, observed, observed_total, "gender")

12015.826627279788

In [305]:
spacygtest(nlp, observed, observed_total, "was")

45.37968197668325

In [306]:
scored = [(item, spacygtest(nlp, observed, observed_total, item)) for item in observed.keys()]
sorted(scored, key=lambda x: x[1], reverse=True)[:50]

[(',', 254620.65645460796),
 ('.', 111284.59791144452),
 (')', 103345.18776314077),
 ('(', 102334.391167828),
 ('-', 56355.1467997031),
 (';', 52298.97169022237),
 ('"', 36411.11024478534),
 ('shall', 30068.883205866343),
 ('school', 26268.65358089468),
 (':', 26124.021149957327),
 ('1', 25105.752805134045),
 ("'s", 22495.992909919172),
 ('2', 21367.405664784354),
 ('section', 17529.315541835225),
 ('sex', 17268.761876729764),
 ('health', 15647.397777237275),
 ('person', 15415.75452638435),
 ('3', 15097.23700207506),
 ('child', 14715.003685784382),
 ('state', 13707.233667194228),
 ('may', 13660.897189952426),
 ('care', 13029.543304133704),
 ('public', 12091.134459433259),
 ('gender', 12015.826627279788),
 ('means', 11726.176410992408),
 ('student', 10480.596001143047),
 ('4', 10440.039936323388),
 ('5', 9652.065342162343),
 ('Section', 9536.182236195595),
 ('medical', 9304.412605062873),
 ('individual', 9292.824004247153),
 ('parent', 9188.526087537852),
 ('SECTION', 8748.147058246008)

---

### Writing to a file

You might want to export lists of words or other things that you make with spaCy to a file, so that you can bring them into other Python programs (or just other programs that form a part of your workflow). One way to do this is to write each item to a single line in a text file. The code in the following cell does exactly this for the word list that we just created:

In [307]:
with open("spacywords.txt", "w") as fh:
    fh.write("\n".join([w.text for w in words]))

The following cell defines a function that performs this for any list of spaCy values you pass to it:

In [308]:
def save_spacy_list(filename, t):
    with open(filename, "w") as fh:
        fh.write("\n".join([item.text for item in t]))

Here's how to use it:

In [309]:
save_spacy_list("spacywords.txt", words)

Since we're working with Counter objects a bunch in this notebook, it makes sense to find a way to save these as files too. The following cell defines a function for writing data from a `Counter` object to a file. The file is in "tab-separated values" format, which you can open using most spreadsheet programs. Execute it before you continue:

In [310]:
def save_counter_tsv(filename, counter, limit=1000):
    with open(filename, "w") as outfile:
        outfile.write("key\tvalue\n")
        for item, count in counter.most_common():
            outfile.write(item.strip() + "\t" + str(count) + "\n")    

Now, run the following cell. You'll end up with a file in the same directory as this notebook called `100_common_words.tsv` that has two columns, one for the words and one for their associated counts:

In [311]:
save_counter_tsv("100_common_words.tsv", word_count, 100)

Try opening this file in Excel or Google Docs or Numbers!

If you want to write the data from another `Counter` object to a file:

* Change the filename to whatever you want (though you should probably keep the `.tsv` extension)
* Replace `word_count` with the name of any of the `Counter` objects we've made in this sheet and use it in place of `word_count`
* Change the number to the number of rows you want to include in your spreadsheet.

### When do things happen in this text?

Here's another example. Using the `times` entities, we can make a spreadsheet of how often particular "times" (durations, times of day, etc.) are mentioned in the text.

In [312]:
time_counter = Counter([e.text.lower().strip() for e in times])
save_counter_tsv("time_count.tsv", time_counter, 100)

Do the same thing, but with people:

In [313]:
people_counter = Counter([e.text.lower() for e in people])
save_counter_tsv("people_count.tsv", people_counter, 100)

## More about words

The list of words that we made above is actually a list of spaCy [Token](https://spacy.io/docs/api/token) objects, which have several interesting attributes. The `.text` attribute gives the  text of the word (as a Python string), and the `.lemma_` attribute gives the word's "lemma" (explained below):

In [314]:
for word in random.sample(words, 12):
    print(word.text, "→", word.lemma_)

manner → manner
chapter → chapter
agency → agency
licensed → license
surgery → surgery
governmental → governmental
gender → gender
pursuant → pursuant
make → make
set → set
permits → permit
within → within


A word's "lemma" is its most "basic" form, the form without any morphology
applied to it. "Sing," "sang," "singing," are all different "forms" of the
lemma *sing*. Likewise, "octopi" is the plural of "octopus"; the "lemma" of
"octopi" is *octopus*.

"Lemmatizing" a text is the process of going through the text and replacing
each word with its lemma. This is often done in an attempt to reduce a text
to its most "essential" meaning, by eliminating pesky things like verb tense
and noun number.

Individual sentences can also be iterated over to get a list of words in that sentence:

In [315]:
sentence = random.choice(sentences)
for word in sentence:
    print(word.text)

Fmt
6652
Sfmt
6201
E:\BILLS\S187.IS


### Parts of speech

Token objects are tagged with their part of speech. For whatever reason, spaCy gives you this part of speech information in two different formats. The `pos_` attribute gives the part of speech using the [universal POS tag](https://universaldependencies.org/u/pos/) system, while the `tag_` attribute gives a more specific designation, using the [Penn Treebank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) system. (Models for different languages will use different schemes; consult the [documentation for your model](https://spacy.io/models) for more information). We used this attribute earlier in the notebook to extract lists of words that had particular parts of speech, but you can access the attribute in other contexts as well:

In [316]:
for item in random.sample(words, 24):
    print(item.text, "/", item.pos_, "/", item.tag_)

Sec / PROPN / NNP
b / NOUN / NN
July / PROPN / NNP
facilitate / VERB / VBP
entity / NOUN / NN
party / NOUN / NN
declared / VERB / VBD
payer / PROPN / NNP
Fmt / NOUN / NN
biological / ADJ / JJ
course / NOUN / NN
merging / VERB / VBG
accordance / NOUN / NN
action / NOUN / NN
c / NOUN / NN
child / NOUN / NN
responsibilities / NOUN / NNS
board / NOUN / NN
provides / VERB / VBZ
context / NOUN / NN
depiction / NOUN / NN
unless / SCONJ / IN
without / ADP / IN
club / NOUN / NN


The `spacy.explain()` function also gives information about what part of speech tags mean:

In [317]:
spacy.explain('VBP')

'verb, non-3rd person singular present'

In [318]:
spacy.explain('HYPH')

'punctuation mark, hyphen'

In [319]:
# Get a set of all the item tags in the corpus
item_tags = set([token.tag_ for token in doc])

# Print the item tags
print(item_tags)

for item in item_tags:
    print(item, spacy.explain(item))

{'JJ', 'JJS', 'FW', '$', 'NFP', 'POS', 'RP', 'UH', 'PDT', 'RB', '.', 'VBZ', 'VBG', 'VB', 'CC', "''", 'ADD', 'PRP$', 'JJR', 'HYPH', '``', 'XX', ',', 'TO', 'DT', 'PRP', 'IN', 'CD', 'NN', 'NNS', '-LRB-', 'RBR', 'MD', 'WDT', 'WRB', 'EX', 'AFX', '-RRB-', 'NNP', 'VBD', 'VBP', 'NNPS', ':', 'SYM', 'VBN', 'LS', 'WP$', '_SP', 'RBS', 'WP'}
JJ adjective (English), other noun-modifier (Chinese)
JJS adjective, superlative
FW foreign word
$ symbol, currency
NFP superfluous punctuation
POS possessive ending
RP adverb, particle
UH interjection
PDT predeterminer
RB adverb
. punctuation mark, sentence closer
VBZ verb, 3rd person singular present
VBG verb, gerund or present participle
VB verb, base form
CC conjunction, coordinating
'' closing quotation mark
ADD email
PRP$ pronoun, possessive
JJR adjective, comparative
HYPH punctuation mark, hyphen
`` opening quotation mark
XX unknown
, punctuation mark, comma
TO infinitival "to"
DT determiner
PRP pronoun, personal
IN conjunction, subordinating or preposit

### Specific forms with `.tag_`

The `.pos_` attribute only gives us general information about the part of speech. The `.tag_` attribute allows us to be more specific about the kinds of verbs we want. For example, this code gives us only the verbs in past participle form:

In [320]:
pronouns = set()
for item in doc:
    if(
        item.tag_ == "PRP$" or 
        item.tag_ == "PRP" or 
        item.tag_ == "WP" or 
        item.tag_ == "WP$"):
        pronouns.add(item)
pronouns

{whose,
 whose,
 us,
 them,
 it,
 who,
 it,
 whose,
 they,
 his,
 her,
 his,
 her,
 his,
 her,
 his,
 her,
 his,
 her,
 whose,
 whose,
 it,
 whose,
 whose,
 her,
 his,
 her,
 his,
 her,
 his,
 her,
 his,
 her,
 she,
 whose,
 whose,
 whose,
 whose,
 whose,
 whose,
 whose,
 whose,
 her,
 who,
 her,
 it,
 it,
 it,
 it,
 it,
 whose,
 whose,
 they,
 it,
 it,
 whose,
 it,
 whose,
 whose,
 whose,
 whose,
 one,
 it,
 who,
 whose,
 it,
 it,
 they,
 us,
 them,
 it,
 it,
 whose,
 whose,
 them,
 one,
 them,
 whose,
 they,
 they,
 they,
 it,
 it,
 it,
 it,
 they,
 who,
 it,
 who,
 it,
 it,
 them,
 it,
 who,
 it,
 it,
 they,
 whose,
 whose,
 they,
 them,
 themselves,
 herself,
 it,
 it,
 it,
 My,
 whose,
 whose,
 himself,
 they,
 it,
 it,
 whose,
 whose,
 whose,
 whose,
 whose,
 whose,
 whose,
 whose,
 THAT12,
 POTENTIALLY,
 CAUSED,
 MUST,
 it,
 it,
 who,
 who,
 whose,
 who,
 whose,
 who,
 who,
 I,
 who,
 it,
 it,
 it,
 our,
 it,
 it,
 it,
 whose,
 it,
 it,
 what,
 what,
 they,
 whose,
 it,
 Act.2,


In [321]:
only_words = [item.text for item in doc if item.tag_ == "NN"]
random.sample(only_words, 12)
# only_words

['setting',
 'advertising',
 'teacher',
 'section',
 'state',
 'wealth',
 'level',
 'gender',
 'loco',
 'faith',
 'communication',
 'perception']

In [ ]:
#JJ, JJS, RP, UH, VBZ, VBG, VB, CC, PRP

In [323]:
only_past = [item.text for item in doc if item.tag_ == 'VBN']

In [324]:
random.sample(only_past, 12)

['deposited',
 'administered',
 'rendered',
 'signed',
 'considered',
 'employed',
 'purchased',
 'MISSISSIPPI',
 'contested',
 'provided',
 'given',
 'defined']

In [ ]:
only_past

Or only plural nouns:

In [326]:
only_plural = [item.text for item in doc if item.tag_ == 'NNS']

In [328]:
random.sample(only_plural, 12)

['changes',
 'materials',
 'substitutes',
 'services',
 'records',
 'buttocks',
 'objects',
 'males',
 'parents',
 'persons',
 'acts',
 'feelings']

## Larger syntactic units

Okay, so we can get individual words and small phrases, like named entities and noun chunks. Great! But what if we want larger chunks, based on their syntactic role in the sentence? For this, we'll need to learn about how spaCy parses sentences into its syntactic components.

### Understanding dependency grammars

The spaCy library parses the underlying sentences using a [dependency grammar](https://en.wikipedia.org/wiki/Dependency_grammar). Dependency grammars look different from the kinds of sentence diagramming you may have done in high school, and even from tree-based [phrase structure grammars](https://en.wikipedia.org/wiki/Phrase_structure_grammar) commonly used in descriptive linguistics. The idea of a dependency grammar is that every word in a sentence is a "dependent" of some other word, which is that word's "head." Those "head" words are in turn dependents of other words. The finite verb in the sentence is the ultimate "head" of the sentence, and is not itself dependent on any other word. The dependents of a particular head are sometimes called its "children."

The question of how to know what constitutes a "head" and a "dependent" is complicated. For more details, consult [Dependency Grammar and Dependency Parsing](http://stp.lingfil.uu.se/~nivre/docs/05133.pdf). But here are some simple guidelines:

* A head determines the syntactic category of a phrase as a whole, and can often stand in for the phrase.
* The meaning of a phrase comes primarily from its head. The dependents further specify the meaning, but don't determine it.
* Heads are obligatory, while dependents are often optional.

For example, in the sentence "Large contented bears hibernate peacefully," *bears* is the head (a noun in this case) and *large* and *contented* are dependents (adjectives). The head of the phrase *large contented bears* is a noun, so the entire phrase is a noun. You could also rewrite the sentence to omit the dependents altogether, and it would still make sense: "Bears hibernate peacefully." Likewise, the adverb *peacefully* is a dependent of the head *hibernate*; the sentence could be rewritten as simply "Bears hibernate." 

Dependents are related to their heads by a *syntactic relation*. The name of the syntactic relation describes the relationship between the head and the dependent. Every token object in a spaCy document or sentence has attributes that tell you what the word's head is, what the dependency relationship is between that word and its head, and a list of that word's children (dependents).

The developers of spaCy included a little tool for visualizing the dependency relations of a particular sentence. Let's look at the sentence "I have eaten the plums that were in the icebox" as an example:

In [329]:
spacy.displacy.render(nlp("I have eaten the plums that were in the icebox."), style='dep')

The arcs you see originate at a head and terminate at dependencies (or children). If you follow all of the arcs back from dependent to head, you'll eventually get back to *eaten*, which is the *root* of the sentence. Each arc is labelled with the dependency *relation*, which tells us what role the dependent fills in the syntax and meaning of the parent word. For example, *I* is related to *eaten* by the `nsubj` relation, which means that *I* is the "nominal subject" of the verb. The word *icebox* is related to the head *in* via the `pobj` relation, meaning that *icebox* is the object of the preposition *in* An exhaustive list of the meanings of these relations can be found in the [Stanford Dependencies Manual](https://downloads.cs.stanford.edu/nlp/software/dependencies_manual.pdf).

The following code prints out each word in the sentence, the tag, the word's head, the word's dependency relation with its head, and the word's children (i.e., dependent words). (This code isn't especially useful on its own, it's just here to help show you how this functionality works.)

In [330]:
sent = random.choice(sentences)
print("Original sentence:", sent.text.replace("\n", " "))
for word in sent:
    print()
    print("Word:", word.text)
    print("Tag:", word.tag_)
    print("Head:", word.head.text)
    print("Dependency relation:", word.dep_)
    print("Children:", list(word.children))

Original sentence: 24 (c) contract employment executed, negotiated, renegotiated July 1, 2023, local school district board directors superintendent meet requirements established section shall null void.

Word: 24
Tag: CD
Head: employment
Dependency relation: nummod
Children: []

Word: (
Tag: -LRB-
Head: employment
Dependency relation: punct
Children: [c]

Word: c
Tag: NN
Head: (
Dependency relation: preconj
Children: []

Word: )
Tag: -RRB-
Head: employment
Dependency relation: punct
Children: []

Word: contract
Tag: NN
Head: employment
Dependency relation: compound
Children: []

Word: employment
Tag: NN
Head: executed
Dependency relation: nsubj
Children: [24, (, ), contract]

Word: executed
Tag: VBN
Head: established
Dependency relation: advcl
Children: [employment, ,, negotiated]

Word: ,
Tag: ,
Head: executed
Dependency relation: punct
Children: []

Word: negotiated
Tag: VBN
Head: executed
Dependency relation: conj
Children: [,, renegotiated]

Word: ,
Tag: ,
Head: negotiated
Dependen

Here's a list of a few dependency relations and what they mean, for quick reference:

* `nsubj`: this word's head is a verb, and this word is itself the subject of the verb
* `nsubjpass`: same as above, but for subjects in sentences in the passive voice
* `dobj`: this word's head is a verb, and this word is itself the direct object of the verb
* `iobj`: same as above, but indirect object
* `aux`: this word's head is a verb, and this word is an "auxiliary" verb (like "have", "will", "be")
* `attr`: this word's head is a copula (like "to be"), and this is the description attributed to the subject of the sentence (e.g., in "This product is a global brand", `brand` is dependent on `is` with the `attr` dependency relation)
* `det`: this word's head is a noun, and this word is a determiner of that noun (like "the," "this," etc.)
* `amod`: this word's head is a noun, and this word is an adjective describing that noun
* `prep`: this word is a preposition that modifies its head
* `pobj`: this word is a dependent (object) of a preposition

### Using .subtree for extracting syntactic units

That's all pretty abstract, so let's get a bit more concrete, and write some code that will let us extract syntactic units based on their dependency relation. There are a couple of things we need in order to do this. The `.subtree` attribute I used in the code above evaluates to a generator that can be flatted by passing it to `list()`. This is a list of the word's syntactic dependents—essentially, the "clause" that the word belongs to.

This function merges a subtree and returns a string with the text of the words contained in it:

In [331]:
def flatten_subtree(st):
    return ''.join([w.text_with_ws for w in list(st)]).strip()

With this function in our toolbox, we can write a loop that prints out the subtree for each word in a sentence. (Again, this code is just here to demonstrate what the process of grabbing subtrees looks like—it doesn't do anything useful yet!)

In [332]:
sent = random.choice(sentences)
print("Original sentence:", sent.text.replace("\n", " "))
for word in sent:
    print()
    print("Word:", word.text.replace("\n", " "))
    print("Flattened subtree: ", flatten_subtree(word.subtree).replace("\n", " "))

Original sentence: (4) Sovereign, governmental qualified immunities suit liability waived abolished extent liability created act.

Word: (
Flattened subtree:  (

Word: 4
Flattened subtree:  (4

Word: )
Flattened subtree:  )

Word: Sovereign
Flattened subtree:  Sovereign

Word: ,
Flattened subtree:  ,

Word: governmental
Flattened subtree:  governmental

Word: qualified
Flattened subtree:  governmental qualified

Word: immunities
Flattened subtree:  Sovereign, governmental qualified immunities

Word: suit
Flattened subtree:  (4) Sovereign, governmental qualified immunities suit liability waived

Word: liability
Flattened subtree:  liability waived

Word: waived
Flattened subtree:  waived

Word: abolished
Flattened subtree:  (4) Sovereign, governmental qualified immunities suit liability waived abolished extent liability created act.

Word: extent
Flattened subtree:  extent

Word: liability
Flattened subtree:  extent liability created act

Word: created
Flattened subtree:  created act

W

Using the subtree and our knowledge of dependency relation types, we can write code that extracts larger syntactic units based on their relationship with the rest of the sentence. For example, to get all of the noun phrases that are subjects of a verb:

In [333]:
subjects = []
for word in doc:
    if word.dep_ in ('nsubj', 'nsubjpass'):
        subjects.append(flatten_subtree(word.subtree))

In [334]:
random.sample(subjects, 12)

[') Division Higher Education',
 'subsequent judgment support arrears',
 'service',
 'grievance related acts practices',
 ': 14 (a) dominant theme conduct, taken whole,',
 'No',
 '1.17 (A)',
 "parent's written consent parent's child",
 'board',
 'every person',
 '(r)(18) licensee',
 'school employees parents, legal guardians,']

Or every prepositional phrase:

In [335]:
prep_phrases = []
for word in doc:
    if word.dep_ == 'prep':
        prep_phrases.append(flatten_subtree(word.subtree).replace("\n", " "))

In [336]:
random.sample(prep_phrases, 12)

['(26',
 'of, basis of',
 '(c)',
 'including,',
 '-153',
 'including limited bruising, lacerations, hematomas, welts, permanent 19 temporary disfigurement impairment bodily function organ',
 '-300',
 'v. Rhode Island Interscholastic 26 League, Inc., 612 A.2d 734, 738 (R.I. 1992)',
 'except sailing, shooting, riding,',
 'except rare cases ambiguous genitalia',
 'including, limited to, castration, vasectomy, hysterectomy, oophorectomy, orchiectomy, penectomy;',
 '-625']

## Generating text from extracted units

One thing I like to do is put together text from parts we've disarticulated with spaCy. Let's use Tracery to do this. If you don't know how to use Tracery, feel free to consult [my Tracery tutorial](tracery-and-python.ipynb) before continuing.

So I want to generate sentences based on things that I've extracted from my text. My first idea: get subjects of sentences, verbs of sentences, nouns and adjectives, and prepositional phrases:

In [337]:
subjects = [flatten_subtree(word.subtree).replace("\n", " ")
            for word in doc if word.dep_ in ('nsubj', 'nsubjpass')]
past_tense_verbs = [word.text for word in words if word.tag_ == 'VBD' and word.lemma_ != 'be']
tense_verbs = [word.text for word in words if word.tag_ == 'VB']
adjectives = [word.text for word in words if word.tag_.startswith('JJ')]
nouns = [word.text for word in words if word.tag_.startswith('NN')]
prep_phrases = [flatten_subtree(word.subtree).replace("\n", " ")
                for word in doc if word.dep_ == 'prep']

Notes on the code above:

* The `.replace("\n", " ")` is in there because spaCy treats linebreaks as normal whitespace, and retains them when we ask for the span's text. For formatting reasons, we want to get rid of this.
* I'm using `.startswith()` in the checks for parts of speech in order to capture other related parts of speech (e.g., `JJR` is comparative adjectives, `NNS` is plural nouns).
* I use only past tense verbs so we don't have to worry about subject/verb agreement in English. I'm excluding forms of *to be* because it is the only verb that agrees with its subject in the past tense.

Now I'll import Tracery. If you haven't already installed it, you can do so using the following cell:

In [379]:
subjects 

['Sixty-eighth Legislative Assembly North Dakota',
 'Subsection 6 section',
 '14-02.4-02 North Dakota Century Code amended reenacted',
 'Discriminatory practice',
 'a. act',
 'status',
 "lawful activity employer's premises",
 'unequal treatment separation segregation persons,',
 'term "discriminate"',
 'Sexual harassment',
 '1526 Sixty-eighth Legislative Assembly favors, sexually motivated physical conduct verbal physical conduct communication sexual nature when: a. (1) Submission conduct communication made term condition, either explicitly implicitly, obtaining employment, public accommodations public services, education;',
 'a. (1) Submission conduct communication',
 'b. (2) Submission rejection',
 'c. (3)',
 'supervisory employees',
 'existence harassment',
 'new section chapter 14-02.4 North Dakota Century Code created enacted',
 '1. used section:',
 'a. "Bodily contact sports',
 'major activity',
 'Sex',
 "biological state female male, based individual's nonambiguous sex organs, c

In [387]:
# subjects
# past_tense_verbs
# tense_verbs
# adjectives
# nouns
# prep_phrases

with open(r'prep_phrases.txt', 'w') as fp:
    for item in prep_phrases:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [338]:
import sys
!{sys.executable} -m pip install tracery

In [339]:
import tracery
from tracery.modifiers import base_english

... and define a grammar. The "trick" of this example is that I grab entire rule expansions from the units extracted from the text using spaCy. The grammar itself is built around producing sentences that look and feel like English.

In [340]:
actverbs = ['CONCERNING', 'ENTITLED', 'ESTABLISHING', 'RELATING TO', 'REQUIRING', 'TO AMEND', 'TO BE KNOWN AS', 'TO ENACT', 'TO ESTABLISH', 'TO PROTECT', 'TO PROVIDE', 'TO REPEAL', 'TO REQUIRE', 'TO RESTRICT']
actfollow = random.sample(actverbs, 1)

In [341]:
actfollow[0]

'CONCERNING'

In [388]:
acts=[]

In [594]:
actverbs = ['CONCERNING', 'ENTITLED', 'ESTABLISHING', 'RELATING TO', 'REQUIRING', 'TO AMEND', 'TO BE KNOWN AS', 'TO ENACT', 'TO ESTABLISH', 'TO PROTECT', 'TO PROVIDE', 'TO REPEAL', 'TO REQUIRE', 'TO RESTRICT']
actfollow = random.sample(actverbs, 1)

conjunctions = ['and', 'or']
conj = random.sample(conjunctions, 1)

rules = {
    "origin": [
        "#nounphrase# #predicate#",
        "#adj.a# #noun#",
        "#nounphrase# #conj# #nounphrase# #predicate#"
    ],
    "predicate": [
        "to #verbb#",
        "to #verbb# #nounphrase#",
        "to #verbb# #prepphrase#"
    ],
    "nounphrase": [
        "the #noun#",
        "the #adj# #noun#",
        "the #noun# #prepphrase#",
        "the #noun# and the #noun#",
        "#noun.a#",
        "#adj.a# #noun#",
        "the #noun# that #predicate#"
    ],
    "subject": subjects,
    "verb": past_tense_verbs,
    "verbb": tense_verbs,
    "noun": nouns,
    "adj": adjectives,
    "prepphrase": prep_phrases,
    "conj": conjunctions
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
actline = grammar.flatten(f"AN ACT {actfollow[0]} #origin#").upper()
acts.append(actline)

In [595]:
len(acts)

200

In [596]:
with open(r'acts.txt', 'w') as fp:
    for item in acts:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [343]:
rules = {
    "origin": [
        "#subject.capitalize# #predicate#.",
        "#subject.capitalize# #predicate#.",
        "#prepphrase.capitalize#, #subject# #predicate#."
    ],
    "predicate": [
        "#verb#",
        "#verb# #nounphrase#",
        "#verb# #prepphrase#"
    ],
    "nounphrase": [
        "the #noun#",
        "the #adj# #noun#",
        "the #noun# #prepphrase#",
        "the #noun# and the #noun#",
        "#noun.a#",
        "#adj.a# #noun#",
        "the #noun# that #predicate#"
    ],
    "subject": subjects,
    "verb": past_tense_verbs,
    "noun": nouns,
    "adj": adjectives,
    "prepphrase": prep_phrases
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.flatten("#origin#")

'Safety maintained the mental women.'

Let's generate a whole paragraph of this and format it nicely:

In [344]:
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(12)])
print(fill(output, 60))

SECTION 4. act authorized. HOUSE BILL No. 1589 BILL ACT
corrected from. -12, 001049 - 1 - ACT managed without
demonstrating burden required compelling governmental
interest. Exception persons depicted the economics that
committed. (J) waived. Concerning definition "sex offense"
Sex Offender Registration Act 1997, c. school institution
higher education brought for. Throughout individual's
minority parent, : 24 (1) juvenile court added an employee.
(g) board provided the environmental drugs. Beginning
2023-2024 school year, (iv) agency records engaged the
research whether without consideration. Approved delegated
to: 28 **** 68th Legislature 2023 SB 458.1 - 6 - Authorized
Print Version – SB 458 (i) public school districts, defined
20-6-101 20-6-701; 1 (ii) special education cooperatives,.
Discernibly turgid state provided. Following, Treatment
section include]Except Testified the section.


I like this approach for a number of reasons. Because I'm using a hand-written grammar, I have a great deal of control over the shape and rhythm of the sentences that are generated. But spaCy lets me pre-populate my grammar's vocabulary without having to write each item by hand.

## Further reading and resources

We've barely scratched the surface of what it's possible to do with spaCy. [The official site has a good list of guides to various natural language processing tasks](https://spacy.io/usage) that you should check out, and there are also [a handful of books that dig deeper into using spaCy for natural language processing tasks](https://spacy.io/universe/category/books).

In [ ]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

pattern = [{"LOWER": "his"}, {"LOWER": "or"}, {"LOWER": "her"}]
matcher.add("HisOrHer", [pattern])

matches = matcher(doc)

if matches:
    match_id, start, end = matches[0]
    next_word = doc[end]
    print(next_word)

In [ ]:
matches

In [ ]:
phrase = "his or her"
window_size = 2 # number of words to the left and right to include

matcher = Matcher(nlp.vocab)
pattern = [{"LOWER": token.lower_} for token in nlp(phrase)]
matcher.add("PhraseMatch", [pattern])

matches = matcher(doc)
words = []

for match_id, start, end in matches:
    # get the left and right neighbors of the phrase
    left_index = max(start - window_size, 0)
    right_index = min(end + window_size + 1, len(doc))
    for token in doc[left_index:end]:
        if token.lower_ != phrase:
            words.append(token.lower_)

# count the occurrence of each word and print the top 10 most common ones
word_counts = Counter(words)
top_words = word_counts.most_common(10)
print(top_words)